In [1]:
# !pip install --upgrade huggingface_hub langchain

In [2]:
from langchain import HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.chains import (
    LLMChain,
    SequentialChain
)

In [3]:
# Globals
repo_id = 'google/flan-t5-xxl'
api_key = '?'
llm = HuggingFaceHub(
    huggingfacehub_api_token=api_key,
    repo_id=repo_id,
    model_kwargs={'temperature': 0.7, 'max_new_tokens': 1000}
)


In [4]:
prompt = PromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

chain = LLMChain(llm=llm, prompt=prompt)

In [5]:
product = "Artistic and visually stunning hand bags."
result = chain.run(product)
print(result)

designer handbags


---

## SequentialChain

In [6]:
# prompt template 1: translate to english
first_prompt = PromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(
    llm=llm, prompt=first_prompt,
    output_key="english_review"
)

second_prompt = PromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence using your own words:"
    "\n\n{english_review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(
    llm=llm, prompt=second_prompt,
    output_key="summary"
)

# prompt template 3: translate to english
third_prompt = PromptTemplate.from_template(
    "What language is the following review:\n\n{review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(
    llm=llm, prompt=third_prompt,
    output_key="language"
)

# prompt template 4: follow up message
fourth_prompt = PromptTemplate.from_template(
    "Is the sentiment in the summary Positive or Negative?"
    "\n\nFeedback: {summary}"
)
# chain 4: input= summary, language and output= sentiment
chain_four = LLMChain(
    llm=llm, prompt=fourth_prompt,
    output_key="sentiment"
)

In [7]:
# overall_chain: input= Review 
# and output= English_Review,summary, sentiment
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["language", "english_review", "summary", "sentiment"],
    verbose=False
)

spanish_review = "Encuentro el sabor mediocre. La espuma no aguanta, es raro. Compro lo mismo en el mercado y el sabor es mucho mejor...\n¿Lote antiguo o falsificado?"

result = overall_chain(spanish_review)
print(result)

{'review': 'Encuentro el sabor mediocre. La espuma no aguanta, es raro. Compro lo mismo en el mercado y el sabor es mucho mejor...\n¿Lote antiguo o falsificado?', 'language': 'Spanish', 'english_review': "I don't like the taste. It doesn't last for a long time. The texture is weird. You can buy the same thing in the market and it tastes much better... Is it an old lot or a fake?", 'summary': "I don't like it.", 'sentiment': 'Negative'}
